In [1]:
#Имена входного и выходного файла
FILENAME = 'in.xlsx'
FILENAME_OUT = 'out.xlsx'

In [8]:
import pandas as pd
import numpy as np

#Прочитать данные из файла Excel
DATA = pd.read_excel(f"{FILENAME}", dtype = str)

#список вопросов
#questions = DATA.iloc[0, 1:].values
questions = DATA.columns[1:].values

#переименовать столбцы (данные из 2 строки)
#for i, name in  enumerate(DATA.columns):
#    if i >= 6:
#        old = name
#        new = DATA.loc[0][i]
#        DATA.rename(columns = {old : new}, inplace = True)

#удалить первую строку
#DATA.drop(index = 0, inplace = True)

#удалить строки с пустой колонкой "Время финиша"
#DATA_ED = DATA.dropna(subset = 'Время финиша')
#DATA_ED = DATA_ED.replace({np.nan: 'Nan'})
DATA_ED = DATA

#Записать в Excel
with pd.ExcelWriter(f"{FILENAME_OUT}", engine = 'xlsxwriter') as writer:

    
    for i, vopros in enumerate(questions):
        vopros_num = vopros.split()[0]
        
        filter_col = [col for col in DATA_ED if col.startswith(vopros_num)]
        vopros_count = len(DATA_ED[filter_col].columns)
        
        
        if vopros_count != 1:
            #если вопрос - составной
            DATA_EDS = DATA_ED[filter_col]
            DATA_EDS = DATA_EDS.dropna(axis = 0, how='all', inplace = False)           
            DATA_EDS = DATA_EDS.replace({np.nan: 'Nan'})
            
            
            total_answers_count = DATA_EDS[vopros].count()
            
            srez = DATA_EDS[vopros].value_counts()
        else:
            total_answers_count = DATA_ED[vopros].count()

            srez = DATA_ED[vopros].value_counts()

        srez_pd = pd.DataFrame(srez)
        srez_pd = srez_pd.sort_index()
        srez_pd.loc['Всего ответов'] = total_answers_count

        srez_pd['Процент'] = (srez_pd[vopros] / total_answers_count)

        #srez_pd
        #srez_pd.to_excel(f"{i}.xlsx", index=True,) #save to xls file
        srez_pd.to_excel(writer, sheet_name = f"{i}_{vopros[:28]}")
        print(f"{i} Запись в файл, вопрос {vopros}")

0 Запись в файл, вопрос 20. Насколько вы довольны услугами продвижения?
1 Запись в файл, вопрос 21. Почему вы поставили такую оценку? Расскажите максимально подробно, пожалуйста.
2 Запись в файл, вопрос 22. Непонятны алгоритмы работы
3 Запись в файл, вопрос 22. Мне не нравятся новые услуги продвижения. Старые были лучше
4 Запись в файл, вопрос 22. Испытываю проблемы с возвратом средств за продвижение при блокировке объявления
5 Запись в файл, вопрос 22. Нет роста просмотров
6 Запись в файл, вопрос 22. Есть прирост просмотров, нет звонков/сообщений
7 Запись в файл, вопрос 22. Есть прирост просмотров и звонков/сообщений, нет продаж
8 Запись в файл, вопрос 22. После применения услуги не вижу поднятия в поиске / объявление очень низко
9 Запись в файл, вопрос 22. Не вижу разницы между услугами разной силы или длительности
10 Запись в файл, вопрос 22. Результат то есть, то нет
11 Запись в файл, вопрос 22. Думаю, что просмотры Авито не настоящие
12 Запись в файл, вопрос 22. Дорого относительн